In [10]:
from bs4 import BeautifulSoup
import urllib
import unicodedata
import re

In [11]:
#read hyperlinks of plays 
links = []
with open('comedy-links.txt', 'r') as f:
    for line in f.readlines():
        if str(line)[0] != '-':
            links.append(str(line).strip())
    

# url = 'http://comediatheque.net/le-joker/'
# url = 'http://comediatheque.net/le-comptoir/'

webpages = []
for url in links:
    webpages.append(urllib.urlopen(url).read())

In [12]:
def parse_webpage(webpage):
    
    soup = BeautifulSoup(webpage, 'html.parser')

    all_pTags = soup.find_all('p')

    ind = 0
    for i, s in enumerate(all_pTags):
        #find beginning of play
        found = False
        for strong in s.find_all('strong'):
            if u'TEXTE INTÉGRAL' == s.string or u'TEXTE DE LA PIÈCE À LIRE OU IMPRIMER' == s.string:
                ind = i
                found = True
        if found:
            break

    # useful_pTags contains <p> tags related to the play
    useful_pTags = all_pTags[ind+1:]


    # print str(useful_pTags)
    def parse_dialog(dialog, speakers):
        parts = dialog.split(' – ')

        raw_speaker = re.sub(r"\(.*\)", "", parts[0])
        speaker = raw_speaker.strip()

        utt = parts[1]

        if speaker in speakers:
            speaker = speakers[speaker]
        else:
            speakers[speaker] = len(speakers) + 1
            speaker = speakers[speaker]

        return speaker, utt


    dialogs = []
    current_dialog = []
    speakers = {}

    for t in useful_pTags:
        # process every <p> tag

        ut_s = ''
        foundMarkup = []
        for ut in t.contents:
            if ut.name == 'strong':
                foundMarkup.append('strong')

            elif ut.name == 'a':
                #skip hyperlinks
                foundMarkup.append('a')

            elif ut.name == 'em':
                foundMarkup.append('em')

        #collapse all contained strings in one
        for s in t.stripped_strings:    
            long_s = str(s.encode('utf8'))
            #remove parenthesis contents
            long_s = re.sub(r"\(.*\)", "", long_s)

            ut_s += long_s

        if 'a' in foundMarkup:
            continue
        #skip strong elements as they are not dialogue
        elif 'strong' in foundMarkup:
            dialogs.append(current_dialog)
            current_dialog = []
        elif ' – ' in ut_s:
            if ' – ISBN' in ut_s:
                #end of document
                break
            result = parse_dialog(ut_s, speakers)
            current_dialog.append(result)

        elif 'em' in foundMarkup:
            #end conversations on emphasis non dialog text of length
            if len(current_dialog) > 2:
                dialogs.append(current_dialog)
                current_dialog = []
    dialogs.append(current_dialog)         
    print len(speakers)
    return dialogs
    # print speakers


In [13]:
plays = []
for page in webpages:
    plays.append(parse_webpage(page))

4
4
6
5
3
4
2
0
4
0
4
0
0
3
3
5
5
6
8
5
5
5
5
6
7
6
5
6
5
5
7
6
5
6
9
6
6
8
7
6
14
8
2
11
12
15
11
10
0
13
9
16
9
17
15
12
15
14
16
20
18
43
17


In [14]:
result = '<dialog>\n'
total_conversations = 0
total_utt = 0

for play in plays:
    nb_conversations = 0
    nb_utt = 0

    for dialog in play:
        if len(dialog) == 0:
            continue
        result += '\t<s>\n'
        nb_conversations += 1
        for i, utt in dialog:
            nb_utt += 1

            t_result = '\t\t<utt uid="' + str(i) + '">' + utt + '</utt>\n'
            result += t_result

        result += '\t</s>\n'
        
    total_conversations += nb_conversations
    total_utt += nb_utt
    if nb_conversations > 0:
        avg = float(nb_utt)/float(nb_conversations)
    else:
        avg = 'n/a'
    print '', nb_conversations, ', ', nb_utt, ', ', avg

if total_conversations > 0:
    avg = float(total_utt)/float(total_conversations)
else:
    avg = 'n/a'
print 'Total, , '
print '', total_conversations,', ', total_utt, ', ', avg

result += '</dialog>'

with open('result.xml', 'w') as f:
#     f.write(str(speakers))
    f.write(result)
    
# text is a list of strings (no <p> or any HTML tags in these strings)

 41 ,  380 ,  9.26829268293
 43 ,  483 ,  11.2325581395
 37 ,  613 ,  16.5675675676
 61 ,  300 ,  4.91803278689
 42 ,  437 ,  10.4047619048
 67 ,  596 ,  8.89552238806
 114 ,  647 ,  5.67543859649
 0 ,  0 ,  n/a
 91 ,  554 ,  6.08791208791
 0 ,  0 ,  n/a
 60 ,  521 ,  8.68333333333
 0 ,  0 ,  n/a
 0 ,  0 ,  n/a
 64 ,  536 ,  8.375
 41 ,  384 ,  9.36585365854
 97 ,  419 ,  4.31958762887
 121 ,  558 ,  4.61157024793
 90 ,  891 ,  9.9
 69 ,  456 ,  6.60869565217
 67 ,  496 ,  7.40298507463
 129 ,  585 ,  4.53488372093
 63 ,  566 ,  8.98412698413
 57 ,  456 ,  8.0
 74 ,  675 ,  9.12162162162
 56 ,  514 ,  9.17857142857
 67 ,  653 ,  9.74626865672
 67 ,  718 ,  10.7164179104
 55 ,  454 ,  8.25454545455
 117 ,  1005 ,  8.58974358974
 73 ,  617 ,  8.45205479452
 53 ,  479 ,  9.03773584906
 90 ,  394 ,  4.37777777778
 72 ,  571 ,  7.93055555556
 110 ,  690 ,  6.27272727273
 73 ,  531 ,  7.27397260274
 79 ,  644 ,  8.15189873418
 66 ,  607 ,  9.19696969697
 86 ,  714 ,  8.3023255814
 72 ,  686 